In [1]:
import pandas as pd
import PIL
from PIL import Image
import tensorflow as tf
from IPython.display import Image as ShowImage
from matplotlib import pyplot as plt
path='C:/Users/EXTREME-PC/Desktop/Zindi/Spot_the_mask/'
import os
from keras.preprocessing.image import img_to_array
from torchvision import transforms
import numpy as np

In [2]:
train = pd.read_csv(path+'train_labels.csv')

train_images_list = train['image'].tolist()

images_list = os.listdir(path+'images/')

test_images_list = [fn for fn in images_list if fn not in train_images_list]

In [3]:
sub=pd.DataFrame({
    'image':test_images_list,
     'target':0
})

In [4]:
image_transform=transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224)
    ])

In [5]:
train_img=[]
for i in range(0,train.shape[0]) :
    img_path="C:/Users/EXTREME-PC/Zindi/Spot The Mask Learning/Spot-the-Mask-Challenge-by_ZindiWeekendz-master/images/"+str(train.iloc[i,:].image)
    image = Image.open(img_path).convert('RGB')
    image=image_transform(image)
    image = np.array(image)
    train_img.append((image))

In [7]:
target = np.array(train.target)
train_img=np.array(train_img)

In [12]:
train_img=train_img/255

In [31]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3),input_shape=(224,224,3),activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512,activation='relu'),
        tf.keras.layers.Dense(units=1,activation='sigmoid')
    ])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [48]:
model.fit(train_img,target,epochs=5)

Epoch 1/5
 6/41 [===>..........................] - ETA: 18s - loss: 5.6100e-04 - accuracy: 1.0000

KeyboardInterrupt: 

In [43]:
test_img=[]
for i in range(0,sub.shape[0]) :
    img_path="C:/Users/EXTREME-PC/Zindi/Spot The Mask Learning/Spot-the-Mask-Challenge-by_ZindiWeekendz-master/images/"+str(sub.iloc[i,:].image)
    image = Image.open(img_path).convert('RGB')
    image=image_transform(image)
    image = np.array(image)
    test_img.append((image))

In [44]:
test_img=np.array(test_img)/255

In [45]:
predictions=model.predict(test_img)

In [46]:
sub.target=predictions

In [47]:
sub.to_csv('C:/Users/EXTREME-PC/Desktop/CNN_Scratch_masks.csv',index=False)